In [23]:
! rm -rf gpHSP
!git clone https://github.com/aspuru-guzik-group/gpHSP

Cloning into 'gpHSP'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 84 (delta 24), reused 37 (delta 15), pack-reused 38
Unpacking objects: 100% (84/84), done.


# Assuming in a colab enviroment

In [ ]:
import sys
if 'google.colab' in sys.modules:
    print('In colab!')
    !pip install rdkit-pypi mordred ml_collections ngboost gpflow
    sys.path.insert(0,'gpHSP')

In [25]:
import pandas as pd
import os
import numpy as np
import gphsp

## Utilities

In [26]:
at_data_dir = lambda x: os.path.join('gpHSP/data', x)

# Load data and get all smiles

In [28]:
files = [at_data_dir('Solvents_exp.csv'),
         at_data_dir('Polymers_exp.csv'),
         at_data_dir('sample_molecules.csv'),
         at_data_dir('HSPiP.csv')
         ]
all_smiles = []
for fname in files:
    df = pd.read_csv(fname)
    df['smiles'] = df['smiles'].apply(gphsp.get_isomeric_smiles)
    gphsp.peek_df(df)
    all_smiles.extend(df['smiles'].tolist())

all_smiles = np.unique(all_smiles)
len(all_smiles)

Index(['key', 'Type', 'δd', 'δp', 'δh', 'smiles', 'ID_type', 'ID', 'Ref',
       'organic', 'n_electrons', 'n_atoms', 'charge', 'MolWt', 'label',
       'finished', 'job_name', 'homo', 'lumo', 'gap', 'd-moments', 'dipole',
       'polar', 'run_time', 'n_cores', 'compute_time', 'Area', 'Hba', 'Hbd',
       'Volume', 'sigma_mom_0', 'sigma_mom_1', 'sigma_mom_2', 'sigma_mom_3',
       'sigma_mom_4', 'sigma_mom_5', 'sigma_norm', 'sigma_profile', 'drug'],
      dtype='object')
(193, 39)


,key,Type,δd,δp,δh,smiles,ID_type,ID,Ref,organic,n_electrons,n_atoms,charge,MolWt,label,finished,job_name,homo,lumo,gap,d-moments,dipole,polar,run_time,n_cores,compute_time,Area,Hba,Hbd,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile,drug
0,"1,1,1-Trichloroethane",Solvent,16.8,4.3,2.0,CC(Cl)(Cl)Cl,CAS,71-55-6,1,True,32,5,0,133.405,Exp-0,True,SOLSPE_Exp-0,-7.096702,-1.107639,5.989063,[-2.37335554 -0.11768285 0.07442233],2.377437,64.37432,94.214,4,376.856,1.3258,0.0,0.0,0.1262,0,16.6,-3.835673,4.240154,-2.020584,1.539736,41.428082,[ 0. 0. 0. 0. ...,False


Index(['label', 'δd', 'δp', 'δh', 'smiles', 'test', 'poly_label', 'n_copies',
       'organic', 'n_electrons', 'n_atoms', 'charge', 'MolWt', 'n_frags',
       'largest', 'finished', 'job_name', 'poly_smiles', 'homo', 'lumo', 'gap',
       'd-moments', 'dipole', 'polar', 'run_time', 'n_cores', 'compute_time',
       'Area', 'Hba', 'Hbd', 'Volume', 'sigma_mom_0', 'sigma_mom_1',
       'sigma_mom_2', 'sigma_mom_3', 'sigma_mom_4', 'sigma_mom_5',
       'sigma_norm', 'sigma_profile'],
      dtype='object')
(31, 39)


,label,δd,δp,δh,smiles,test,poly_label,n_copies,organic,n_electrons,n_atoms,charge,MolWt,n_frags,largest,finished,job_name,poly_smiles,homo,lumo,gap,d-moments,dipole,polar,run_time,n_cores,compute_time,Area,Hba,Hbd,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile
0,Polyacrylonitrile-n5,20.0,15.1,7.9,CCC#N,True,Polyacrylonitrile,5,True,110,45,0,275.4,1,True,True,SOLSPE2_Polyacrylonitrile-n5,[C@@H](C#N)(CCC#N)C[C@@H](C#N)C[C@H](C#N)CCC#N,-7.879002,-1.120048,6.758954,[ 0.58302575 -8.49403297 -1.86749721],8.716425,211.62387,468.596,4,1874.384,3.047,1.5386,0.2196,0.3427,0,175.0,26.1,154.0,51.4,165.0,66.770346,[ 0. 0. 0. 0. ...


Index(['names', 'smiles'], dtype='object')
(11, 2)


,names,smiles
0,J52-Cl,CCCCCCCC[C@@H](CCCCCC)Cn1nc2c(-c3ccc(C)s3)c(F)...


Index(['Chemical', 'CAS', 'smiles', 'HSPiP_indx', 'good_smiles', 'single_frag',
       'duplicated', 'sanity', 'label', 'organic', 'n_electrons', 'n_atoms',
       'charge', 'MolWt', 'homo', 'lumo', 'gap', 'd-moments', 'dipole',
       'polar', 'run_time', 'n_cores', 'compute_time', 'Area', 'Hba', 'Hbd',
       'Volume', 'sigma_mom_0', 'sigma_mom_1', 'sigma_mom_2', 'sigma_mom_3',
       'sigma_mom_4', 'sigma_mom_5', 'sigma_norm', 'sigma_profile'],
      dtype='object')
(8852, 35)


,Chemical,CAS,smiles,HSPiP_indx,good_smiles,single_frag,duplicated,sanity,label,organic,n_electrons,n_atoms,charge,MolWt,homo,lumo,gap,d-moments,dipole,polar,run_time,n_cores,compute_time,Area,Hba,Hbd,Volume,sigma_mom_0,sigma_mom_1,sigma_mom_2,sigma_mom_3,sigma_mom_4,sigma_mom_5,sigma_norm,sigma_profile
0,Acetaldehyde,75-07-0,CC=O,0,True,True,True,True,hspip_0,True,18,7,0,44.053,-5.76279991450395,-1.924389,3.838411,[-2.36128225 1.996745 0.05889226],3.092913,31.05401,26.178,1,26.178,0.8298,0.6278,0.0,0.0641,0,33.0,18.4,30.6,30.1,39.1,22.203331,[ 0.00000000e+00 0.00000000e+00 0.0000000...


8833

Calculate a mordred values and a mask

In [30]:
all_smiles = np.unique([gphsp.get_isomeric_smiles(s) for s in all_smiles])
all_values = gphsp.calculate_mordred(all_smiles)
mask = gphsp.calculate_mask(all_values)
np.savez_compressed(at_data_dir('mordred_features.npz'),
                    smiles = all_smiles,
                    values = all_values,
                    mask = mask)
features = gphsp.SmilesMap(at_data_dir('mordred_features.npz'))
features(all_values)

  2%|▏         | 194/8833 [00:38<26:53,  5.35it/s]  

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 11%|█         | 954/8833 [03:08<46:43,  2.81it/s]

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 8833/8833 [32:28<00:00,  4.53it/s]
